In [1]:
!pip install langchain chromadb  tiktoken  wikipedia

  Using cached tiktoken-0.9.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.7 kB)
  Preparing metadata (setup.py) ... done
Using cached tiktoken-0.9.0-cp312-cp312-macosx_11_0_arm64.whl (1.0 MB)
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=74b8b73ceea10234faf5c8e63c8c20e352bf3fa71d4a85d39bd95967817a98d1
  Stored in directory: /Users/subhadyutirath/Library/Caches/pip/wheels/63/47/7c/a9688349aa74d228ce0a9023229c6c0ac52ca2a40fe87679b8
Successfully built wikipedia


In [2]:
from langchain_community.retrievers import WikipediaRetriever

In [3]:
retriever = WikipediaRetriever(top_k_results=2, lang="en")

In [6]:
query = "History of Rohit Sharma in Mumbai Indians"

# Get relevant Wikipedia documents
docs = retriever.invoke(query)

In [20]:
# Print the summary from each document in the docs list
for doc in docs:
	print(doc.metadata['summary'])

Rohit Sharma (born 30 April 1987) is an Indian international cricketer and the captain of the national team in ODIs. He is also a former captain in Tests and T20Is. He is widely regarded as one of the greatest T20I opening batters of all time. He is a right-handed batsman who plays for Mumbai Indians in Indian Premier League and for Mumbai in domestic cricket. After the team's victory at the 2024 Men's T20 World Cup, he announced his retirement from T20Is. He was also a member of the teams that won the 2007 T20 World Cup, the 2013 ICC Champions Trophy and the 2025 ICC Champions Trophy.
Sharma holds several batting records which famously include most runs in T20 Internationals, most sixes in international cricket, most double centuries in ODI cricket (3), most centuries at Cricket World Cups (7) and joint most hundreds in Twenty20 Internationals (5).He also holds the world record for the highest individual score (264) in a One Day International (ODI) and also holds the record for scorin

In [9]:
# Vector Store Retrievers
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [10]:
documents = [
    Document(page_content="LangChain helps developers build LLM applications easily."),
    Document(page_content="Chroma is a vector database optimized for LLM-based search."),
    Document(page_content="Embeddings convert text into high-dimensional vectors."),
    Document(page_content="OpenAI provides powerful embedding models."),
]

In [11]:
# Step 2: Initialize embedding model
embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Step 3: Create Chroma vector store in memory
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    collection_name="my_collection"
)

In [12]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

In [18]:
query = "What is Chroma used for?"
results = retriever.invoke(query, k=1)
print(results[0].page_content)

Chroma is a vector database optimized for LLM-based search.


In [22]:
# Maximum Marginal Retrievers
from langchain_community.vectorstores import FAISS

In [32]:
# Sample documents
docs = [
    Document(page_content="LangChain makes it easy to work with LLMs."),
    Document(page_content="LangChain is used to build LLM based applications."),
    Document(page_content="Chroma is used to store and search document embeddings."),
    Document(page_content="Embeddings are vector representations of text."),
    Document(page_content="MMR helps you get diverse results when doing similarity search."),
    Document(page_content="LangChain supports Chroma, FAISS, Pinecone, and more."),
]

In [24]:
pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 9.2 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")


vectorstore = FAISS.from_documents(
    documents=docs,
    embedding=embedding_model
)

In [ ]:
retriever = vectorstore.as_retriever(
    search_type="mmr",                   # <-- This enables MMR
    search_kwargs={"k": 3, "lambda_mult": 0.5}  # k = top results, lambda_mult = relevance-diversity balance
    
    # lambda mult = 1 -> Similarity Search
    # lambda mult = 0 -> Diverse Results
)

In [41]:
query = "What is langchain?"
results = retriever.invoke(query)
print(results)

[Document(id='7d15c451-7a02-4906-bd24-4e7eabe3b4d6', metadata={}, page_content='LangChain supports Chroma, FAISS, Pinecone, and more.'), Document(id='d69ad1c8-6b94-4efe-be4d-8703554436f0', metadata={}, page_content='LangChain is used to build LLM based applications.'), Document(id='604675d8-8af7-47dd-af31-15717d75095d', metadata={}, page_content='Embeddings are vector representations of text.')]


In [43]:
# Multi-Query Retriever
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_google_genai import ChatGoogleGenerativeAI
import os

In [44]:
model = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    api_key=os.getenv("GOOGLE_API_KEY")
)

In [ ]:
all_docs = [
    Document(page_content="Regular walking boosts heart health and can reduce symptoms of depression.", metadata={"source": "H1"}),
    Document(page_content="Consuming leafy greens and fruits helps detox the body and improve longevity.", metadata={"source": "H2"}),
    Document(page_content="Deep sleep is crucial for cellular repair and emotional regulation.", metadata={"source": "H3"}),
    Document(page_content="Mindfulness and controlled breathing lower cortisol and improve mental clarity.", metadata={"source": "H4"}),
    Document(page_content="Drinking sufficient water throughout the day helps maintain metabolism and energy.", metadata={"source": "H5"}),
    Document(page_content="The solar energy system in modern homes helps balance electricity demand.", metadata={"source": "I1"}),
    Document(page_content="Python balances readability with power, making it a popular system design language.", metadata={"source": "I2"}),
    Document(page_content="Photosynthesis enables plants to produce energy by converting sunlight.", metadata={"source": "I3"}),
    Document(page_content="The 2022 FIFA World Cup was held in Qatar and drew global energy and excitement.", metadata={"source": "I4"}),
    Document(page_content="Black holes bend spacetime and store immense gravitational energy.", metadata={"source": "I5"}),
]

In [47]:
vectorstore = FAISS.from_documents(documents=all_docs, embedding=embedding_model)

In [51]:
similarity_retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [48]:
multiquery_retriever = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(search_kwargs={"k": 5}),
    llm=model
)

In [50]:
query = "How to improve energy levels and maintain balance?"

In [52]:
similarity_results = similarity_retriever.invoke(query)
multiquery_results= multiquery_retriever.invoke(query)

In [53]:
print(similarity_results)
print(multiquery_results)

[Document(id='26fa6172-3d80-49e5-93eb-6040af07a476', metadata={'source': 'H5'}, page_content='Drinking sufficient water throughout the day helps maintain metabolism and energy.'), Document(id='77b4fac4-fe67-4874-92a3-b65161b555ea', metadata={'source': 'H4'}, page_content='Mindfulness and controlled breathing lower cortisol and improve mental clarity.'), Document(id='58ea9aa4-aea3-48d6-87c3-64081fc8b2cd', metadata={'source': 'H2'}, page_content='Consuming leafy greens and fruits helps detox the body and improve longevity.'), Document(id='886ea988-746d-46df-ba2e-d2d19a2eda5e', metadata={'source': 'H3'}, page_content='Deep sleep is crucial for cellular repair and emotional regulation.'), Document(id='a78feaf9-b932-4add-baea-647ecf1a90d8', metadata={'source': 'H1'}, page_content='Regular walking boosts heart health and can reduce symptoms of depression.')]
[Document(id='26fa6172-3d80-49e5-93eb-6040af07a476', metadata={'source': 'H5'}, page_content='Drinking sufficient water throughout the 

In [54]:
# Contextual Compression Retriever
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [ ]:
docs = [
    Document(page_content=(
        """The Grand Canyon is one of the most visited natural wonders in the world.
        Photosynthesis is the process by which green plants convert sunlight into energy.
        Millions of tourists travel to see it every year. The rocks date back millions of years."""
    ), metadata={"source": "Doc1"}),

    Document(page_content=(
        """In medieval Europe, castles were built primarily for defense.
        The chlorophyll in plant cells captures sunlight during photosynthesis.
        Knights wore armor made of metal. Siege weapons were often used to breach castle walls."""
    ), metadata={"source": "Doc2"}),

    Document(page_content=(
        """Basketball was invented by Dr. James Naismith in the late 19th century.
        It was originally played with a soccer ball and peach baskets. NBA is now a global league."""
    ), metadata={"source": "Doc3"}),

    Document(page_content=(
        """The history of cinema began in the late 1800s. Silent films were the earliest form.
        Thomas Edison was among the pioneers. Photosynthesis does not occur in animal cells.
        Modern filmmaking involves complex CGI and sound design."""
    ), metadata={"source": "Doc4"})
]

In [56]:
vectorstore = FAISS.from_documents(docs, embedding_model)

In [57]:
base_retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
llm = model
compressor = LLMChainExtractor.from_llm(llm)

In [58]:
compression_retriever = ContextualCompressionRetriever(
    base_retriever=base_retriever,
    base_compressor=compressor
)

In [59]:
query = "What is photosynthesis?"
compressed_results = compression_retriever.invoke(query)

In [60]:
print(compressed_results)

[Document(metadata={'source': 'Doc4'}, page_content='Photosynthesis does not occur in animal cells.'), Document(metadata={'source': 'Doc1'}, page_content='Photosynthesis is the process by which green plants convert sunlight into energy.'), Document(metadata={'source': 'Doc2'}, page_content='The chlorophyll in plant cells captures sunlight during photosynthesis.')]
